# Tensorflow Categorical Encoding

---
# Setup

In [1]:
!pip install tensorflow_decision_forests
!pip install wurlitzer

In [2]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from wurlitzer import sys_pipes


tf.__version__

'2.6.0'

## undefined symbol: _ZNK10tensorflow8OpKernel11TraceStringERKNS_15OpKernelContextEb

* [NotFoundError: undefined symbol: _ZNK10tensorflow8OpKernel11TraceStringERKNS_15OpKernelContextEb #65](https://github.com/tensorflow/decision-forests/issues/65)

* [Tensorflow: undefined symbol: _ZN10tensorflow11GetNodeAttrERKNS_9AttrSliceEN4absl14lts_2020_09_2311string_viewEPSs
](https://stackoverflow.com/questions/68834328/tensorflow-undefined-symbol-zn10tensorflow11getnodeattrerkns-9attrsliceen4abs)  
The exact same issue of TFDF following the same steps.

> Try to uninstall tensorflow (in mine colab it was 2.6.0) install tf 2.5.0 and then install tensorflow_decision_forests.

* [tensorflow.python.framework.errors_impl.NotFoundError: /usr/local/lib/python3.6/dist-packages/tensorflow_text/python/metrics/_text_similarity_metric_ops.so: undefined symbol: _ZN10tensorflow8OpKernel11TraceStringEPNS_15OpKernelContextEb #385](https://github.com/tensorflow/text/issues/385)

In [3]:
import tensorflow_decision_forests as tfdf

# Data

In [4]:
train, info = tfds.load(
    'titanic:2.*.*',              # Name of the dataset
    with_info=True,       # Information of the dataset
    shuffle_files=True, 
    split='train[:90%]'
)
validation = tfds.load(
    'titanic:2.*.*',              # Name of the dataset
    with_info=False,       # Information of the dataset
    shuffle_files=True, 
    split='train[:10%]'
)
info

tfds.core.DatasetInfo(
    name='titanic',
    version=2.0.0,
    description='Dataset describing the survival status of individual passengers on the Titanic. Missing values in the original dataset are represented using ?. Float and int missing values are replaced with -1, string missing values are replaced with 'Unknown'.',
    homepage='https://www.openml.org/d/40945',
    features=FeaturesDict({
        'features': FeaturesDict({
            'age': tf.float32,
            'boat': tf.string,
            'body': tf.int32,
            'cabin': tf.string,
            'embarked': ClassLabel(shape=(), dtype=tf.int64, num_classes=4),
            'fare': tf.float32,
            'home.dest': tf.string,
            'name': tf.string,
            'parch': tf.int32,
            'pclass': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
            'sex': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
            'sibsp': tf.int32,
            'ticket': tf.string,
        }),
        '

In [5]:
train = train.map(lambda row: (row['features'], row['survived']))
validation = validation.map(lambda row: (row['features'], row['survived']))
tf.data.experimental.get_structure(train)

({'age': TensorSpec(shape=(), dtype=tf.float32, name=None),
  'boat': TensorSpec(shape=(), dtype=tf.string, name=None),
  'body': TensorSpec(shape=(), dtype=tf.int32, name=None),
  'cabin': TensorSpec(shape=(), dtype=tf.string, name=None),
  'embarked': TensorSpec(shape=(), dtype=tf.int64, name=None),
  'fare': TensorSpec(shape=(), dtype=tf.float32, name=None),
  'home.dest': TensorSpec(shape=(), dtype=tf.string, name=None),
  'name': TensorSpec(shape=(), dtype=tf.string, name=None),
  'parch': TensorSpec(shape=(), dtype=tf.int32, name=None),
  'pclass': TensorSpec(shape=(), dtype=tf.int64, name=None),
  'sex': TensorSpec(shape=(), dtype=tf.int64, name=None),
  'sibsp': TensorSpec(shape=(), dtype=tf.int32, name=None),
  'ticket': TensorSpec(shape=(), dtype=tf.string, name=None)},
 TensorSpec(shape=(), dtype=tf.int64, name=None))

## Examin dataset

In [6]:
[(train_features, label_batch)] = train.batch(5).take(1)
print('Features:', list(train_features.keys()))
for feature in train_features.keys():
    print(f'A batch of {feature}:', train_features[feature])
print('A batch of targets:', label_batch )

Features: ['age', 'boat', 'body', 'cabin', 'embarked', 'fare', 'home.dest', 'name', 'parch', 'pclass', 'sex', 'sibsp', 'ticket']
A batch of age: tf.Tensor([30. 37. 28. 18. -1.], shape=(5,), dtype=float32)
A batch of boat: tf.Tensor([b'Unknown' b'Unknown' b'9' b'Unknown' b'Unknown'], shape=(5,), dtype=string)
A batch of body: tf.Tensor([-1 98 -1 -1 -1], shape=(5,), dtype=int32)
A batch of cabin: tf.Tensor([b'Unknown' b'Unknown' b'Unknown' b'Unknown' b'Unknown'], shape=(5,), dtype=string)
A batch of embarked: tf.Tensor([2 2 2 2 0], shape=(5,), dtype=int64)
A batch of fare: tf.Tensor([13.      7.925  13.     73.5     7.8958], shape=(5,), dtype=float32)
A batch of home.dest: tf.Tensor(
[b'Sarnia, ON' b'Ruotsinphytaa, Finland New York, NY' b'Spain'
 b'Lyndhurst, England' b'Unknown'], shape=(5,), dtype=string)
A batch of name: tf.Tensor(
[b'McCrie, Mr. James Matthew' b'Gustafsson, Mr. Anders Vilhelm'
 b'Reynaldo, Ms. Encarnacion' b'Davies, Mr. Charles Henry'
 b'Gheorgheff, Mr. Stanio'], shap

---
# Training

In [7]:
batch_size = 32
train = train.batch(batch_size).shuffle(buffer_size=32).prefetch(1)
validation = validation.batch(batch_size).shuffle(buffer_size=32).prefetch(1)

## TFDF RandomForest

* [tfdf.keras.RandomForestModel](https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel)
Training algorithms do not need validation datasets. If a validation dataset is provided, it will only be used to show metrics.

In [8]:
features = []
for feature in ['age', 'boat', 'cabin', 'embarked', 'fare' ,'pclass', 'sex', 'sibsp', 'ticket']:
    features.append(tfdf.keras.FeatureUsage(name=feature))

model = tfdf.keras.RandomForestModel(
    task = tfdf.keras.Task.CLASSIFICATION,
    features=features,
    exclude_non_specified_features=True  
)
model.compile(
    metrics=['accuracy']
)

In [9]:
with sys_pipes():
    model.fit(x=train)

37/37 [==============================] - 6s 2ms/step


[INFO kernel.cc:736] Start Yggdrasil model training
[INFO kernel.cc:737] Collect training examples
[INFO kernel.cc:392] Number of batches: 37
[INFO kernel.cc:393] Number of examples: 1178
[INFO data_spec_inference.cc:290] 8 item(s) have been pruned (i.e. they are considered out of dictionary) for the column boat (20 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO data_spec_inference.cc:290] 175 item(s) have been pruned (i.e. they are considered out of dictionary) for the column cabin (3 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO data_spec_inference.cc:290] 839 item(s) have been pruned (i.e. they are considered out of dictionary) for the column ticket (16 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO kernel.cc:759] Dataset:
Number of records: 1178
Number of columns: 10

Number of columns by type:
	NUMERICAL: 6 (60%)
	CATEGORICAL: 4 (40%)

Columns:

NUMERICAL: 6 (60%)
	0: "age"

In [10]:
!mkdir -p model
model.save('model/titanic_random_forest_model')
reloaded_model = tf.keras.models.load_model('model/titanic_random_forest_model')

#del train, model

INFO:tensorflow:Assets written to: model/titanic_random_forest_model/assets


INFO:tensorflow:Assets written to: model/titanic_random_forest_model/assets


# Evaluation



In [11]:
evaluation = model.evaluate(validation)
print(evaluation)

5/5 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 0.9924
[0.0, 0.9923664331436157]
